In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
cd /content/drive/MyDrive/논문/Youtube-Recommendation/Movie-Lens

/content/drive/MyDrive/논문/Youtube-Recommendation/Movie-Lens


In [ ]:
ls

 dataset_item_pca.pkl
 dataset.pkl
 logs/
 meta.tsv
 ml-100k/
 model.png
 MovieLens_Youtube_Recommendation_Candidate_Generation_Network.ipynb
 movielens.zip
 Untitled0.ipynb
 vecs.tsv
'Wine_Candidate_Rec_[Youtube_Recommendation_Candidate_Generation_Network].ipynb'


In [ ]:
import os
import sys
import joblib
import numpy as np
import pandas as pd
from tqdm import tqdm

In [ ]:
data = joblib.load('./dataset_item_pca.pkl')
locals().update(data)

In [ ]:
train

userID  wine_id  rating  ...     pca_48     pca_49     pca_50
0       19484511  2532733     4.0  ...   2.432840   7.759076   5.353219
1       19484511  1253802     3.5  ...   0.246672   3.099425   7.057004
2       19484511  1123441     3.5  ...  -0.625199  -0.121658  -0.512128
3       19484511  1157656     3.5  ...  27.401769 -26.849115 -10.134520
4       19484511  1134756     3.5  ...  -5.353860  -3.307175  -2.662975
...          ...      ...     ...  ...        ...        ...        ...
937751  16607503  1135203     3.5  ...   6.676951  -9.274513  -4.011704
937752  16607503  4303173     3.0  ...  -4.634302  -5.391337  12.593795
937753  16607503  2364090     4.0  ...  -1.217088  -0.792093   5.938541
937754  16607503  1679387     3.5  ...  29.080365 -14.936483 -12.768522
937755  16607503  1838367     4.0  ...   7.636543 -12.046227  -3.981710

[937756 rows x 58 columns]

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()

In [ ]:
minmax = scaler.fit_transform(train.loc[train['userID'] == 19484511, 'rating'].values.reshape(-1, 1))

In [ ]:
train['label'] = None
for u in tqdm(train.userID.unique()):
  minmax = scaler.fit_transform(train.loc[train['userID'] == u, 'rating'].values.reshape(-1, 1))
  avg = np.mean(minmax)

  res = (minmax > np.mean(minmax)).astype(int)
  res = [r[0] for r in res]

  train.loc[train['userID'] == u, 'label'] = res

100%|██████████| 6343/6343 [01:54<00:00, 55.34it/s]


In [ ]:
train.label.value_counts()

1    499407
0    438349
Name: label, dtype: int64

In [ ]:
train = train.sample(frac = 1)

In [ ]:
EMBEDDING_DIMS = 100
DENSE_UNITS = 64
DROPOUT_PCT = 0.0
ALPHA = 0.0
NUM_CLASSES = 50862
LEARNING_RATE = 0.003

In [ ]:
import tensorflow as tf
import keras
class MaskedEmbeddingsAggregatorLayer(tf.keras.layers.Layer):
    def __init__(self, agg_mode='sum', **kwargs):
        super(MaskedEmbeddingsAggregatorLayer, self).__init__(**kwargs)

        if agg_mode not in ['sum', 'mean']:
            raise NotImplementedError('mode {} not implemented!'.format(agg_mode))
        self.agg_mode = agg_mode
    
    @tf.function
    def call(self, inputs, mask=None):
        masked_embeddings = tf.ragged.boolean_mask(inputs, mask)
        if self.agg_mode == 'sum':
            aggregated =  tf.reduce_sum(masked_embeddings, axis=1)
        elif self.agg_mode == 'mean':
            aggregated = tf.reduce_mean(masked_embeddings, axis=1)
        
        return aggregated
    
    def get_config(self):
        # this is used when loading a saved model that uses a custom layer
        return {'agg_mode': self.agg_mode}

In [ ]:
class L2NormLayer(tf.keras.layers.Layer):
    def __init__(self, **kwargs):
        super(L2NormLayer, self).__init__(**kwargs)
    
    @tf.function
    def call(self, inputs, mask=None):
        if mask is not None:
            inputs = tf.ragged.boolean_mask(inputs, mask).to_tensor()
        return tf.math.l2_normalize(inputs, axis=-1)

    def compute_mask(self, inputs, mask):
        return mask
    

In [ ]:
#---inputs
import tensorflow as tf
import datetime
import os
input_profile = tf.keras.Input(shape=(50, ), name='profile')
input_recent = tf.keras.Input(shape=(None, ), name='recent')
# inp_liked = tf.keras.layers.Input(shape=(None,), name='like')
# inp_disliked = tf.keras.layers.Input(shape=(None,), name='dislike')\
inp_relation = tf.keras.layers.Input(shape = (None, ), name = 'relation')


# --- layers
# features_embedding_layer = tf.keras.layers.Embedding(input_dim=NUM_CLASSES, output_dim=EMBEDDING_DIMS, 
#                                             mask_zero=True, trainable=True, name='features_embeddings')
labels_embedding_layer = tf.keras.layers.Embedding(input_dim = NUM_CLASSES, output_dim = EMBEDDING_DIMS, 
                                            mask_zero=True, trainable=True, name='labels_embeddings')

avg_embeddings = MaskedEmbeddingsAggregatorLayer(agg_mode='mean', name='aggregate_embeddings')

dense_1 = tf.keras.layers.Dense(units=256, name='dense_1')
dense_2 = tf.keras.layers.Dense(units=128, name='dense_2')
dense_3 = tf.keras.layers.Dense(units=64, name='dense_3')
l2_norm_1 = L2NormLayer(name='l2_norm_1')

dense_output = tf.keras.layers.Dense(1,
                                     activation=tf.nn.softmax,
                                     name='dense_output')

# --- features

# features_embeddings = features_embedding_layer(input_profile)
# l2_norm_features = l2_norm_1(features_embeddings)
# avg_features = avg_embeddings(l2_norm_features)

dense_profile = tf.keras.layers.Dense(units = 32, name='dense_profile')
dense_1_profile = dense_profile(input_profile)
dense_1_profile_relu = tf.keras.layers.ReLU(name='dense_1_profile_relu')(dense_1_profile)

labels_relation_embeddings = labels_embedding_layer(inp_relation)
l2_norm_relation = l2_norm_1(labels_relation_embeddings)
avg_relation = avg_embeddings(l2_norm_relation)

# labels_liked_embeddings = labels_embedding_layer(inp_liked)
# l2_norm_liked = l2_norm_1(labels_liked_embeddings)
# avg_liked = avg_embeddings(l2_norm_liked)

# labels_disliked_embeddings = labels_embedding_layer(inp_disliked)
# l2_norm_disliked = l2_norm_1(labels_disliked_embeddings)
# avg_disliked = avg_embeddings(l2_norm_disliked)

labels_genre_embeddings = labels_embedding_layer(input_recent)
l2_norm_genre = l2_norm_1(labels_genre_embeddings)
avg_recent = avg_embeddings(l2_norm_genre)



concat_inputs = tf.keras.layers.Concatenate(axis=1)([
                                                    #  avg_features,
                                                     dense_1_profile_relu,
                                                    #  avg_liked,
                                                     avg_relation,
                                                    #  avg_disliked,
                                                     avg_recent
                                                     ])
# Dense Layers
dense_1_features = dense_1(concat_inputs)
dense_1_relu = tf.keras.layers.ReLU(name='dense_1_relu')(dense_1_features)
dense_1_batch_norm = tf.keras.layers.BatchNormalization(name='dense_1_batch_norm')(dense_1_relu)

dense_2_features = dense_2(dense_1_relu)
dense_2_relu = tf.keras.layers.ReLU(name='dense_2_relu')(dense_2_features)
dense_2_batch_norm = tf.keras.layers.BatchNormalization(name='dense_2_batch_norm')(dense_2_relu)

dense_3_features = dense_3(dense_2_relu)
dense_3_relu = tf.keras.layers.ReLU(name='dense_3_relu')(dense_3_features)
dense_3_batch_norm = tf.keras.layers.BatchNormalization(name='dense_3_batch_norm')(dense_3_relu)
outputs = dense_output(dense_3_batch_norm)

#Optimizer
optimiser = tf.keras.optimizers.Adam(learning_rate = LEARNING_RATE)

#--- prep model
model = tf.keras.models.Model(
    inputs=[
            input_profile,
            # inp_liked,
            inp_relation
            # inp_disliked
            ,input_recent
            ],
    outputs=[outputs]
)
logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)
model.compile(optimizer = optimiser,
              loss = tf.keras.losses.BinaryCrossentropy(),
              # loss='mean_squared_error',
              metrics=['acc'])


In [ ]:
tf.keras.utils.plot_model(model, show_shapes=True, show_layer_names=True,dpi=96)

In [ ]:
model.summary()

Model: "model_4"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
relation (InputLayer)           [(None, None)]       0                                            
__________________________________________________________________________________________________
recent (InputLayer)             [(None, None)]       0                                            
__________________________________________________________________________________________________
profile (InputLayer)            [(None, 50)]         0                                            
__________________________________________________________________________________________________
labels_embeddings (Embedding)   (None, None, 100)    5086200     relation[0][0]                   
                                                                 recent[0][0]               

In [ ]:
callbacks_list = [
                  keras.callbacks.EarlyStopping(
                    monitor = 'acc',
                    patience = 10
                  )
]

In [ ]:
train.iloc[:, 8:-1].shape

(937756, 50)

In [ ]:
model.fit([
           train.iloc[:, 8:-1].fillna(0).values,
          #  tf.keras.preprocessing.sequence.pad_sequences(train['like']),
           train[['enc_userID', 'enc_wine_id']].values,
          #  tf.keras.preprocessing.sequence.pad_sequences(train['dislike']),
           tf.keras.preprocessing.sequence.pad_sequences(train['recent'])
           ],
          train['label'].astype(int).values,
          # callbacks=[tensorboard_callback],
          batch_size = 512,
          # steps_per_epoch = 1,
          epochs = 1000,
          # callbacks = callbacks_list,
          # validation_data = ([
          #                     tf.keras.preprocessing.sequence.pad_sequences(test['like']),
          #                     tf.keras.preprocessing.sequence.pad_sequences(test['dislike']),
          #                     tf.keras.preprocessing.sequence.pad_sequences(test['recent'])
          # ], test['rating'].values)
          )

Epoch 1/1000
1832/1832 [==============================] - 107s 58ms/step - loss: 7.1323 - acc: 0.5323
Epoch 2/1000
1832/1832 [==============================] - 106s 58ms/step - loss: 7.1388 - acc: 0.5319
Epoch 3/1000
1832/1832 [==============================] - 106s 58ms/step - loss: 7.1270 - acc: 0.5326
Epoch 4/1000
1832/1832 [==============================] - 105s 58ms/step - loss: 7.1262 - acc: 0.5327
Epoch 5/1000
1832/1832 [==============================] - 108s 59ms/step - loss: 7.1296 - acc: 0.5325
Epoch 6/1000
1832/1832 [==============================] - 107s 58ms/step - loss: 7.1214 - acc: 0.5330
Epoch 7/1000
1832/1832 [==============================] - 107s 58ms/step - loss: 7.1274 - acc: 0.5326
Epoch 8/1000
1832/1832 [==============================] - 110s 60ms/step - loss: 7.1154 - acc: 0.5334
Epoch 9/1000
1832/1832 [==============================] - 109s 60ms/step - loss: 7.1229 - acc: 0.5329
Epoch 10/1000
1832/1832 [==============================] - 110s 60ms/step - loss: 

In [ ]:
predict = model.predict([
                        tf.keras.preprocessing.sequence.pad_sequences(train['like']),
                        tf.keras.preprocessing.sequence.pad_sequences(train['dislike']),
                        tf.keras.preprocessing.sequence.pad_sequences(train['recent'])
])
                                                                  

In [ ]:
pd.Series([p[0] for p in predict]).value_counts()

3.926826    937756
dtype: int64

In [ ]:
predict = model.predict([
                        tf.keras.preprocessing.sequence.pad_sequences(test['like']),
                        tf.keras.preprocessing.sequence.pad_sequences(test['dislike']),
                        tf.keras.preprocessing.sequence.pad_sequences(test['recent')
])
                                                                  

In [ ]:
user_list_1=user_title_list[user_title_list.user>600]

In [ ]:
user_list_1.head(10)

In [ ]:
predict =model.predict([tf.keras.preprocessing.sequence.pad_sequences(user_list_1['title_d']),
          tf.keras.preprocessing.sequence.pad_sequences(user_list_1['like']),
          tf.keras.preprocessing.sequence.pad_sequences(user_list_1['dislike']),
          tf.keras.preprocessing.sequence.pad_sequences(user_list_1['all_genres'])
          ])


In [ ]:
import numpy as np

In [ ]:
predictions = np.array([np.argmax(a) for a in predict])
pd.DataFrame(predictions)

0
0   602
1   602
2   602
3   602
4   602
5   602
6   602
7   602
8   602
9   602
10  602
11  602
12  602
13  602
14  602
15  602
16  602
17  602
18  602
19  602
20  602
21  602
22  602
23  602
24  602
25  602
26  602
27  602
28  602
29  602
30  602
31  602
32  602
33  602
34  602
35  602
36  602
37  602
38  602
39  602
40  602
41  602
42  602
43  602
44  602
45  602
46  602
47  602
48  602
49  602

In [ ]:
dataset2['label']

1        39572
3         6689
6        42949
9        33103
11        2694
         ...  
35747    30653
35786     5478
35792     7551
35793    28711
35796     5119
Name: label, Length: 6229, dtype: int64

NameError: ignored

In [ ]:
user_list_1['predicted_label'] = predictions

In [ ]:
user_list_1

In [ ]:
print(tf.argsort(predict,direction='DESCENDING',axis=-1))

In [ ]:
tf.nn.top_k(
    predict,
    k=20,
    sorted=True,
    name=None
)

In [ ]:
s=[i for i in range(len(movie2movie_encoded)) if movie2movie_encoded[i] == 365 ]

In [ ]:
movies[movies.movie_id == ''.join(str(s[0]))]

### Visualize Embeddings 

In [ ]:
import io
out_v = io.open('vecs.tsv', 'w', encoding='utf-8')
out_m = io.open('meta.tsv', 'w', encoding='utf-8')
#weights =model.layers[4].get_weights()[0]
weights = features_embedding_layer.get_weights()[0][1:]

for num, word in enumerate(title2title_encoded):
  vec = weights[num+1] # skip 0, it's padding.
  out_m.write(word + "\n")
  out_v.write('\t'.join([str(x) for x in vec]) + "\n")
out_v.close()
out_m.close()

NameError: ignored

In [ ]:
try:
  from google.colab import files
except ImportError:
   pass
else:
  files.download('vecs.tsv')
  files.download('meta.tsv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>